In [1]:

import argparse
import torch

import transformers

import datetime
import easydict
import itertools
import json
import matplotlib
import pathlib
import pprint
import re

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from operator import itemgetter
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from typing import List, Dict
import os

In [15]:
config = easydict.EasyDict({

    'gpu_id': 3,
    'train_data_path': '../datasets/integrated_pre_datasets/train_data.tsv',
    'valid_data_path': '../datasets/integrated_pre_datasets/valid_data.tsv',
    'test_data_path': '../datasets/integrated_pre_datasets/test_data.tsv',
    'pretrained_model_name': 'gogamza/kobart-base-v1',
    'ckpt': "ckpt",
    'logs': "logs",
    'batch_size_per_device': 8, # 8이 최선
    'gradient_accumulation_steps': 16,
    'lr': 5e-5,
    'weight_decay': 1e-2,
    'warmup_ratio': .2,
    'n_epochs': 10,
    'inp_max_len': 1024,
    'tar_max_len': 256,
    'model_fpath': "model_records/kogpt2-model.pth",
    'beam_size': 5,
    "prompt4summary": "1줄요약"
})

In [16]:
import torch

saved_data = torch.load(
    "/workspace/home/uglee/Projects/title_extraction/src/model_records/kogpt2-model.pth",
    map_location="cpu" if config.gpu_id < 0 else "cuda:%d" % config.gpu_id
)

In [17]:
kogpt2_best = saved_data["model"]
train_config = saved_data["config"]
tokenizer = transformers.PreTrainedTokenizerFast.from_pretrained(train_config.pretrained_model_name)

## Load weights.
model = transformers.GPT2LMHeadModel.from_pretrained(train_config.pretrained_model_name)
model.resize_token_embeddings( len(tokenizer) )

model.load_state_dict(kogpt2_best)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


<All keys matched successfully>

In [18]:
from dataloaders.gpt2_dataloader import get_datasets
from dataloaders.gpt2_dataloader import TextAbstractSummarizationCollator
from torch.utils.data import DataLoader

ts_ds = get_datasets(
    tokenizer,
    config=config,
    fpath=Path(config.test_data_path), 
    mode="test"
    )

ts_loader = DataLoader(
    ts_ds,
    batch_size=config.batch_size_per_device,
    shuffle=False,
    collate_fn=TextAbstractSummarizationCollator(
        tokenizer=tokenizer,
        config=config
    )
)

Tokenizing input texts: 100%|██████████| 3052/3052 [00:01<00:00, 1889.98it/s]


In [22]:

device = torch.device('cpu') if config.gpu_id < 0 else torch.device('cuda:%d' % config.gpu_id)


with torch.no_grad():
    for idx, data in enumerate(tqdm(ts_loader)):
        model.eval()

        input_ids = data['input_ids']
        attention_mask = data['attention_mask']
        labels = data['labels']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)


        outputs = model(
            input_ids,
            labels=labels,
            attention_mask=attention_mask
        )#.to(self.device)

        outputs = tokenizer.batch_decode(
            outputs.tolist(),
            skip_special_tokens=True
        )

        outputs.extend([{"id": id_, "output": output_} for id_, output_ in zip(id, outputs)])

outputs = sorted(
    outputs,
    key=itemgetter("id"),
    reverse=False,
)
outputs = [i["output"] for i in outputs]

ts <torch.utils.data.dataloader.DataLoader object at 0x7f493f97ceb0>


  0%|          | 0/382 [00:00<?, ?it/s]


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
print(outputs[0])

냉이캐기 냉이 냉이이이기캐기?이네 냉이네이이캐김 냉이 두이이 캐기냉이이냉이캐캐기서 냉이김이이 제이이김기이이제 냉이어이이 냉이가  냉이
